## Initial setup

In [6]:
%matplotlib ipympl

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pykonal
# import seispy

HOME         = os.environ['HOME']
GOOGLE_DRIVE = os.environ['GOOGLE_DRIVE']

# Define a utility function to convert geographical coordinates to spherical
def geo2sph(lat, lon, depth):
    return(
        6371 - depth,
        np.pi/2 - np.radians(lat),
        np.radians(lon)
    )

### Reformat the velocity model and save as NumPy .npz file for fast I/O

This only needs to be done once, but I am leaving it here for you to look at in case you want to reformat a different data set.

In [ ]:
import glob

with open(os.path.join(GOOGLE_DRIVE, 'malcolm.white@usc.edu', 'data', 'velocity', 'Golos_et_al', 'lon.txt')) as inf:
    phi = np.array([float(v) for v in inf.read().split('\n')[:-1]])
    
with open(os.path.join(GOOGLE_DRIVE, 'malcolm.white@usc.edu', 'data', 'velocity', 'Golos_et_al', 'lat.txt')) as inf:
    lamda = np.array([float(v) for v in inf.read().split('\n')[:-1]])
    
velocity = {}
for file in sorted(glob.glob(os.path.join(GOOGLE_DRIVE, 'malcolm.white@usc.edu', 'data', 'velocity', 'Golos_et_al', 'Vs*.txt'))):
    depth = float(os.path.splitext(os.path.basename(file))[0][2:])
    with open(file) as inf:
        velocity[depth] = np.array([float(v) for v in inf.read().split('\n')[:-1]])
        
df0 = pd.DataFrame()
for depth in velocity:
    df          =  pd.DataFrame(velocity[depth], columns=('Vs',))
    df['theta'] = np.pi/2 - lamda
    df['phi']   = phi
    df['rho']   = 6371 - depth
    df0         = df0.append(df, ignore_index=True)
df0 = df0.sort_values(['rho', 'theta', 'phi'])

nrho, ntheta, nphi = len(df0['rho'].unique()), len(df0['theta'].unique()), len(df0['phi'].unique())
drho               = np.mean(np.diff(df0['rho'].unique()))
dtheta             = np.mean(np.diff(df0['theta'].unique()))
dphi               = np.mean(np.diff(df0['phi'].unique()))
rho0               = df0['rho'].min()
theta0             = df0['theta'].min()
phi0               = df0['phi'].min()
vs = df0['Vs'].values.reshape((nrho, ntheta, nphi))

np.savez_compressed(
    os.path.join(GOOGLE_DRIVE, 'malcolm.white@usc.edu', 'data', 'velocity', 'Golos_et_al', 'Vs.npz'),
    Vs=vs,
    min_coords=np.array([rho0, theta0, phi0]),
    node_intervals=np.array([drho, dtheta, dphi]),
    npts=np.array([nrho, ntheta, nphi])
)

# Example 1
-----
### 1.1 Set up and run a simple 3D example
This takes ~1.5-2 minutes on my desktop.

In [2]:
# This is the index of the grid node that I am treating as the source.
# Sources that do not fall exactly on a grid node take a little more set up;
# more to come on that later.
src_idx                        = (100, 63, 124)

# Load the velocity model from the NumPy .npz file
npz = np.load(
    os.path.join(GOOGLE_DRIVE, 'malcolm.white@usc.edu', 'data', 'velocity', 'Golos_et_al', 'Vs.npz'),
)

# Instantiate an pykonal.EikonalSolver object.
far_field                      = pykonal.EikonalSolver(coord_sys='spherical')
# Define the velocity model.
far_field.vgrid.min_coords     = npz['min_coords']
far_field.vgrid.node_intervals = npz['node_intervals']
far_field.vgrid.npts           = npz['npts']
far_field.vv                   = npz['Vs']

# Set the travel time to zero at the source node
far_field.uu[src_idx]     = 0
# And update the appropriate state variables.
far_field.is_far[src_idx] = False
far_field.close.push(*src_idx)

# Solve the Eikonal equation.
%time far_field.solve()

CPU times: user 2min 35s, sys: 1.45 s, total: 2min 37s
Wall time: 2min 36s


## 1.2 Plot the resulting travel-time field
### 1.2.1 Convert spherical coordinates to cartesian for plotting

In [3]:
pgrid = far_field.pgrid[...]
vgrid = far_field.vgrid[...]
uu = far_field.uu
vv = far_field.vv
xxp  = pgrid[...,0] * np.sin(pgrid[...,1]) * np.cos(pgrid[...,2])
yyp  = pgrid[...,0] * np.sin(pgrid[...,1]) * np.sin(pgrid[...,2])
zzp  = pgrid[...,0] * np.cos(pgrid[...,1])
xxv  = vgrid[...,0] * np.sin(vgrid[...,1]) * np.cos(vgrid[...,2])
yyv  = vgrid[...,0] * np.sin(vgrid[...,1]) * np.sin(vgrid[...,2])
zzv  = vgrid[...,0] * np.cos(vgrid[...,1])

### 1.2.2 Plot the travel time field for constant $\phi = \phi_0$

In [7]:
# Choose the index of phi to plot
ip = 0

# Plot the velocity model in the first subplot.
fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1, aspect=1)
qmesh = ax1.pcolormesh(
    xxv[:,:,ip],
    zzv[:,:,ip],
    vv[:,:,ip],
    cmap=plt.get_cmap('jet'),
    shading='gouraud'
)
cbar = fig.colorbar(qmesh, ax=ax1, orientation='horizontal')
cbar.set_label('Velocity [km/s]')
ax1.scatter(
    xxv[src_idx],
    zzv[src_idx],
    s=250,
    marker='*',
    edgecolor='k',
    facecolor='w',
    linewidth=1
)

# Plot the travel-time field in the second subplot.
ax2 = fig.add_subplot(1, 2, 2, aspect=1)
qmesh = ax2.pcolormesh(
    xxp[:,:,ip],
    zzp[:,:,ip],
    uu[:,:,ip],
    cmap=plt.get_cmap('jet_r'),
    shading='gouraud',
    vmax=2e3
)
ax2.yaxis.tick_right()
cbar = fig.colorbar(qmesh, ax=ax2, orientation='horizontal')
cbar.set_label('Travel time [s]')
ax2.scatter(
    xxp[src_idx],
    zzp[src_idx],
    s=250,
    marker='*',
    edgecolor='k',
    facecolor='w',
    linewidth=1
)

FigureCanvasNbAgg()

### 1.2.3 Plot the travel-time field for constant $\theta = \theta_0$

In [8]:
it = src_idx[1]

fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1, aspect=1)
qmesh = ax1.pcolormesh(
    xxv[:,it,:],
    yyv[:,it,:],
    vv[:,it,:],
    cmap=plt.get_cmap('jet'),
    shading='gouraud'
)
cbar = fig.colorbar(qmesh, ax=ax1, orientation='horizontal')
cbar.set_label('Velocity [km/s]')
ax1.scatter(
    xxv[src_idx],
    yyv[src_idx],
    s=250,
    marker='*',
    edgecolor='k',
    facecolor='w',
    linewidth=1
)
ax2 = fig.add_subplot(1, 2, 2, aspect=1)
qmesh = ax2.pcolormesh(
    xxp[:,it,:],
    yyp[:,it,:],
    uu[:,it,:],
    cmap=plt.get_cmap('jet_r'),
    shading='gouraud',
    vmax=uu[:-1,it,:].max()
)
ax2.yaxis.tick_right()
cbar = fig.colorbar(qmesh, ax=ax2, orientation='horizontal')
cbar.set_label('Travel time [s]')
ax2.scatter(
    xxp[src_idx],
    yyp[src_idx],
    s=250,
    marker='*',
    edgecolor='k',
    facecolor='w',
    linewidth=1
)

FigureCanvasNbAgg()

# Example 2
-----

This example uses a refined grid in the source region to improve accuracy.

## 2.1. Set up and run a more involved example
### 2.1.1 Define the source location

In [18]:
# Specify the source coordinates
lat0, lon0, depth0 = 0, 90, 410

# Convert the source coordinates to spherical coordinates
rho0   = 6371. - depth0
theta0 = np.pi/2 - np.radians(lat0)
phi0   = np.radians(lon0)

### 2.1.2 Set up the far-field grid, but don't solve yet

In [19]:
# Full 3D
npz = np.load(
    os.path.join(GOOGLE_DRIVE, 'malcolm.white@usc.edu', 'data', 'velocity', 'Golos_et_al', 'Vs.npz'),
)
far_field                      = pykonal.EikonalSolver(coord_sys='spherical')
far_field.vgrid.min_coords     = npz['min_coords']
far_field.vgrid.node_intervals = npz['node_intervals']
far_field.vgrid.npts           = npz['npts']
far_field.vv                   = npz['Vs']

### 2.1.3 Set up a refined grid in the near-field region
This grid is centered on the source

In [20]:
near_field                      = pykonal.EikonalSolver(coord_sys='spherical')
near_field.vgrid.min_coords     = npz['node_intervals'][0] / 5, 0, 0
near_field.vgrid.node_intervals = npz['node_intervals'][0] / 5, np.pi/40, np.pi/40
near_field.vgrid.npts           = 50, 41, 80

near_field.transfer_velocity_from(far_field, (-rho0, theta0, phi0))

vvi = pykonal.LinearInterpolator3D(near_field.vgrid, near_field.vv)

for it in range(near_field.pgrid.npts[1]):
    for ip in range(near_field.pgrid.npts[2]):
        idx = (0, it, ip)
        near_field.uu[idx]     = near_field.pgrid[idx + (0,)] / vvi(near_field.pgrid[idx])
        near_field.is_far[idx] = False
        near_field.close.push(*idx)
        
%time near_field.solve()

CPU times: user 1.12 s, sys: 2.21 ms, total: 1.13 s
Wall time: 1.13 s


In [21]:
far_field.transfer_travel_times_from(near_field, (-rho0, theta0, phi0), set_alive=True)

In [22]:
%time far_field.solve()

CPU times: user 2min 45s, sys: 1.2 s, total: 2min 46s
Wall time: 2min 46s


## 1.2 Plot the resulting travel-time field
### 1.2.1 Convert spherical coordinates to cartesian for plotting

In [23]:
pgrid = far_field.pgrid[...]
vgrid = far_field.vgrid[...]
uu = far_field.uu
vv = far_field.vv
if far_field.pgrid.is_periodic[2]:
    pgrid = np.append(pgrid, pgrid[:,:,0].reshape(*pgrid.shape[:2], 1, 3), axis=2)
    vgrid = np.append(vgrid, vgrid[:,:,0].reshape(*vgrid.shape[:2], 1, 3), axis=2)
    uu   = np.append(uu, uu[...,0].reshape(*uu.shape[:2], 1), axis=2)
    vv   = np.append(vv, vv[...,0].reshape(*vv.shape[:2], 1), axis=2)
xxp  = pgrid[...,0] * np.sin(pgrid[...,1]) * np.cos(pgrid[...,2])
yyp  = pgrid[...,0] * np.sin(pgrid[...,1]) * np.sin(pgrid[...,2])
zzp  = pgrid[...,0] * np.cos(pgrid[...,1])
xxv  = vgrid[...,0] * np.sin(vgrid[...,1]) * np.cos(vgrid[...,2])
yyv  = vgrid[...,0] * np.sin(vgrid[...,1]) * np.sin(vgrid[...,2])
zzv  = vgrid[...,0] * np.cos(vgrid[...,1])


### 1.2.3 Plot the travel-time field for constant $\theta = \theta_0$

In [49]:
it = 128

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(1, 2, 1, aspect=1)
qmesh = ax1.pcolormesh(
    xxv[:,it,:],
    yyv[:,it,:],
    vv[:,it,:],
    cmap=plt.get_cmap('jet'),
    shading='gouraud'
)
cbar = fig.colorbar(qmesh, ax=ax1, orientation='horizontal')
cbar.set_label('Velocity [km/s]')
ax1.scatter(
    rho0 * np.sin(theta0) * np.cos(phi0),
    rho0 * np.sin(theta0) * np.sin(phi0),
    s=250,
    marker='*',
    edgecolor='k',
    facecolor='w',
    linewidth=1
)
ax2 = fig.add_subplot(1, 2, 2, aspect=1)
qmesh = ax2.pcolormesh(
    xxp[:,it,:],
    yyp[:,it,:],
    uu[:,it,:],
    cmap=plt.get_cmap('jet_r'),
    shading='gouraud',
    vmax=uu[:-1,it,:].max()
)
ax2.yaxis.tick_right()
cbar = fig.colorbar(qmesh, ax=ax2, orientation='horizontal')
cbar.set_label('Travel time [s]')
ax2.scatter(
    rho0 * np.sin(theta0) * np.cos(phi0),
    rho0 * np.sin(theta0) * np.sin(phi0),
    s=250,
    marker='*',
    edgecolor='k',
    facecolor='w',
    linewidth=1
)
for dlon in (-89, -80, -70, -60, -50, -40, -30, -20, -10, 10, 20, 30, 40, 50, 60, 70, 80, 90):
    try:
        ray = far_field.trace_ray(geo2sph(lat0, lon0+dlon, 102))
    except:
        continue
    ax2.plot(
        ray[:,0] * np.sin(ray[:,1]) * np.cos(ray[:,2]),
        ray[:,0] * np.sin(ray[:,1]) * np.sin(ray[:,2]),
        'k',
        linewidth=0.5
    )

FigureCanvasNbAgg()

In [45]:
np.degrees(far_field.pgrid.node_intervals[2])

0.70311945420846467

In [ ]:
uui = lambda lat, lon, depth: pykonal.LinearInterpolator3D(far_field.pgrid, far_field.uu)(geo2sph(lat, lon, depth))

In [36]:
np.unique(far_field.pgrid[:,0,0])

array([  5.79632679e-03,   6.13600000e-03,   3.49079490e+03,
         3.51338474e+03,   3.53597459e+03,   3.55856443e+03,
         3.58115428e+03,   3.60374412e+03,   3.62633396e+03,
         3.64892381e+03,   3.67151365e+03,   3.69410350e+03,
         3.71669334e+03,   3.73928319e+03,   3.76187303e+03,
         3.78446287e+03,   3.80705272e+03,   3.82964256e+03,
         3.85223241e+03,   3.87482225e+03,   3.89741209e+03,
         3.92000194e+03,   3.94259178e+03,   3.96518163e+03,
         3.98777147e+03,   4.01036131e+03,   4.03295116e+03,
         4.05554100e+03,   4.07813085e+03,   4.10072069e+03,
         4.12331053e+03,   4.14590038e+03,   4.16849022e+03,
         4.19108007e+03,   4.21366991e+03,   4.23625976e+03,
         4.25884960e+03,   4.28143944e+03,   4.30402929e+03,
         4.32661913e+03,   4.34920898e+03,   4.37179882e+03,
         4.39438866e+03,   4.41697851e+03,   4.43956835e+03,
         4.46215820e+03,   4.48474804e+03,   4.50733788e+03,
         4.52992773e+03,